In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn



df = pd.read_csv('../data/processed_sunspot_data.csv')
data = df['SunspotNumber'].values

# Normalize the data to the range [0, 1]
min_val = np.min(data)
max_val = np.max(data)
normalized_data = (data - min_val) / (max_val - min_val)

# Create sequences for time series prediction
seq_length = 12  # Using past 12 months to predict the next month
prediction_step = 1  # Predicting 1 month ahead
X, y = [], []
for i in range(len(normalized_data) - seq_length - prediction_step + 1):
    X.append(normalized_data[i:i + seq_length])
    y.append(normalized_data[i + seq_length + prediction_step - 1])
X = np.array(X)
y = np.array(y)

# Split the data into training, validation, and test sets
train_ratio = 0.7
val_ratio = 0.15
N = len(X)
train_size = int(N * train_ratio)
val_size = int(N * val_ratio)
test_size = N - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]   


In [ ]:
class MLPModel(nn.Module):
    def __init__(self, seq_length, hidden_size=64):
        super().__init__()
        self.fc1 = nn.Linear(seq_length, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
class RNNModel(nn.Module):
    def __init__(self, seq_length, hidden_size=32, num_layers=1):
        super().__init__()
        self.rnn = nn.RNN(seq_length, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), 64).to(x.device)
        out, _ = self.rnn(x.unsqueeze(-1), h0)
        out = self.fc(out[:, -1, :])
        return out
    